In [130]:
import io
import sys, os
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt

In [131]:
#SE USAN LAS FUNCIONES EN LA CARPETA MARKOVIANAS PARA TRAER LAS MATRICES DE TRANSICION

#pip install nbformat
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

with HiddenPrints():
    %run -i ../MARKOVIANAS/Ford_Case2.ipynb
    ford_accepted, ford_transition = sendInfoToAnalize()
    %run -i ../MARKOVIANAS/Honda_Case2.ipynb
    honda_accepted, honda_transition = sendInfoToAnalize()
    %run -i ../MARKOVIANAS/NIO_Case2.ipynb
    NIO_accepted, NIO_transition = sendInfoToAnalize()
    %run -i ../MARKOVIANAS/Toyota_Case2.ipynb
    toyota_accepted, toyota_transition = sendInfoToAnalize()

states_map = {"B1":0, "B2":1, "B3":2, "S1":3, "S2":4, "S3": 5} 
company_map= {"Ford": ford_transition, "Honda": honda_transition, "NIO": NIO_transition, "Toyota": toyota_transition}


In [132]:
def calculateProbabilityUpAndDownAfterDays(days,company,initial_state,type="S"):
    if(type=="S"):
        final_type="subida"
    elif(type=="B"):
        final_type="bajada"

    all_combinations = list(itertools.product(list(states_map.keys()),repeat=days+1))
    selected_combinations=[]
    for _,combination in enumerate(all_combinations):
        if all(type in combination[i] for i in range(1, len(combination))) and combination[0]==initial_state:
            selected_combinations.append(combination)
    probabilities=[]

    for _,selected_combination in enumerate(selected_combinations):
        probability=0
        for i in range(len(selected_combination)-1):
            if i==0:
                probability=(company_map[company][states_map[selected_combination[i]],states_map[selected_combination[i+1]]])
            else:
                probability=probability*(company_map[company][states_map[selected_combination[i]],states_map[selected_combination[i+1]]])
        probabilities.append(probability)
    final_probability=sum(probabilities)
    return final_probability


In [133]:
#CALCULAR 3 DÍAS SEGUIDOS DE SUBIDA
days=3
for _,comapany in enumerate(list(company_map.keys())):
    print(f"Empresa: {comapany}")
    list_prob_subida=[]
    for _,state in enumerate(list(states_map.keys())):
        probability=calculateProbabilityUpAndDownAfterDays(days,comapany,state,"S")
        list_prob_subida.append(probability)
    data_frame_prob=pd.DataFrame(list_prob_subida,index=list(states_map.keys()), columns=[f"Probabilidad de S - {days} consecutivos"])
    print(data_frame_prob,"\n")


Empresa: Ford
    Probabilidad de S - 3 consecutivos
B1                            0.138105
B2                            0.143367
B3                            0.097596
S1                            0.141237
S2                            0.139127
S3                            0.142644 

Empresa: Honda
    Probabilidad de S - 3 consecutivos
B1                            0.149968
B2                            0.147660
B3                            0.168147
S1                            0.148690
S2                            0.142684
S3                            0.159311 

Empresa: NIO
    Probabilidad de S - 3 consecutivos
B1                            0.108689
B2                            0.129594
B3                            0.130711
S1                            0.108360
S2                            0.120696
S3                            0.129822 

Empresa: Toyota
    Probabilidad de S - 3 consecutivos
B1                            0.142199
B2                            0.164983


In [134]:
#CALCULAR 3 DÍAS SEGUIDOS DE BAJADA
days=3
for _,comapany in enumerate(list(company_map.keys())):
    print(f"Empresa: {comapany}")
    list_prob_subida=[]
    for _,state in enumerate(list(states_map.keys())):
        probability=calculateProbabilityUpAndDownAfterDays(days,comapany,state,"B")
        list_prob_subida.append(probability)
    data_frame_prob=pd.DataFrame(list_prob_subida,index=list(states_map.keys()), columns=[f"Probabilidad de B - {days} consecutivos"])
    print(data_frame_prob,"\n")

Empresa: Ford
    Probabilidad de B - 3 consecutivos
B1                            0.118526
B2                            0.118021
B3                            0.156092
S1                            0.116158
S2                            0.118048
S3                            0.113486 

Empresa: Honda
    Probabilidad de B - 3 consecutivos
B1                            0.100015
B2                            0.100561
B3                            0.085222
S1                            0.100250
S2                            0.105316
S3                            0.093725 

Empresa: NIO
    Probabilidad de B - 3 consecutivos
B1                            0.136316
B2                            0.109360
B3                            0.116279
S1                            0.138198
S2                            0.122104
S3                            0.119804 

Empresa: Toyota
    Probabilidad de B - 3 consecutivos
B1                            0.097148
B2                            0.082133


In [135]:
def get_state_distribution(P, n):
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    stationary_distribution = np.array(eigenvectors[:, np.isclose(eigenvalues, 1)])
    stationary_distribution = stationary_distribution.real / stationary_distribution.real.sum()
    return stationary_distribution

distribuciones = {}
for company, transition in company_map.items():
    distribuciones[company] = get_state_distribution(transition, 6)
    print(f"Empresa: {company}")
    print(get_state_distribution(transition, 6),"\n")

Empresa: Ford
[[0.35719572]
 [0.10592228]
 [0.02285204]
 [0.35722977]
 [0.11734947]
 [0.03945072]] 

Empresa: Honda
[[0.31889309]
 [0.12245574]
 [0.02593801]
 [0.37483656]
 [0.12670962]
 [0.03116698]] 

Empresa: NIO
[[0.36454768]
 [0.12665971]
 [0.02077893]
 [0.33117055]
 [0.11740082]
 [0.03944231]] 

Empresa: Toyota
[[0.32193078]
 [0.11423595]
 [0.03010747]
 [0.3696167 ]
 [0.12982629]
 [0.03428281]] 



In [136]:
def tiempos_primera_pasada(transicion):
    n = len(transicion)

    matriz_probabilidades_uij = np.zeros((n**2, n**2))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if k != j:
                    matriz_probabilidades_uij[i * n + j][k * n + j] = transicion[i][k]

    matriz_identidad = np.eye(n**2)
    A =  matriz_probabilidades_uij - matriz_identidad
    B = np.ones(shape=(n**2))
    vector_primeras_pasadas = np.linalg.solve(A, -B)
    return vector_primeras_pasadas


def reescribir_diagonal(matriz, states_map=states_map):
    filas = len(matriz)
    columnas = len(matriz[0])
    tiempo_demora = {}
    for i in range(min(filas, columnas)):
        tiempo_demora[i] = matriz[i][i]
        matriz[i][i] = 0
    states_map_invertido = {v: k for k, v in states_map.items()}
    tiempo_demora_estado = {states_map_invertido[i]: tiempo for i, tiempo in tiempo_demora.items()}
    return matriz, tiempo_demora_estado

def imprimir_matrices_primeras_pasadas_tiempo_demora(states_map, company_map):
    for company, transition in company_map.items():
        print(f"Matriz de tiempos de primera pasada para {company}:")
        vector_primeras_pasadas = tiempos_primera_pasada(transition)
        resize_value = int(len(vector_primeras_pasadas)**0.5)
        matriz_primeras_pasadas = vector_primeras_pasadas.reshape((resize_value, resize_value))
        matriz_primeras_pasadas, tiempo_demora = reescribir_diagonal(matriz_primeras_pasadas, states_map)
        print(matriz_primeras_pasadas, "\n")
        print(f"Tiempo de demora para cada estado en {company}:")
        print(tiempo_demora, "\n")
        print()

imprimir_matrices_primeras_pasadas_tiempo_demora(states_map, company_map)

Matriz de tiempos de primera pasada para Ford:
[[ 0.          9.56509517 45.08079036  2.83366284  8.31646296 26.55565346]
 [ 3.02483999  0.         42.63585486  2.96202943  8.12558052 25.67131389]
 [ 2.3996334   9.73926085  0.          3.6237591   6.9636511  27.34415813]
 [ 2.8570405   9.53528    44.79571     0.          8.25220982 26.78274489]
 [ 2.77142341  9.70631184 44.99384932  2.83409851  0.         25.86783587]
 [ 2.89575486  9.28943213 45.70140504  2.88526166  8.40058996  0.        ]] 

Tiempo de demora para cada estado en Ford:
{'B1': 2.799585624637983, 'B2': 9.440884165036755, 'B3': 43.759769074503424, 'S1': 2.7993187672267466, 'S2': 8.521555281983273, 'S3': 25.34808135343661} 


Matriz de tiempos de primera pasada para Honda:
[[ 0.          8.96540913 39.21084589  2.63680331  7.66548017 31.51001819]
 [ 3.49599303  0.         38.33762174  2.58142163  7.52741748 32.91921669]
 [ 3.30833503  9.19474576  0.          2.76354161  7.02506994 30.47054163]
 [ 3.20109417  8.77967847 39

## d

In [137]:
#ford data
ford_data = pd.read_csv('../CSV/Ford2.csv')
ford_data = ford_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])

percentage = [0,]
for i in range(1, len(ford_data)):
    percent = (ford_data['Adj Close'][i] - ford_data['Adj Close'][i-1])/ford_data['Adj Close'][i-1]
    percentage.append(percent)
ford_data.insert(2, 'Change %', percentage)

std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
ford_data.insert(3, 'std dev', std_dev)

states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
ford_data.insert(4, 'Current State', states)

ganancias_compañias = {}
ganancia_promedio_por_estado_F = ford_data.groupby('Current State')['Change %'].mean()
ganancia_promedio_por_estado_F = ganancia_promedio_por_estado_F.to_dict()
ganancia_promedio_por_estado_F = {estado: ganancia for estado, ganancia in ganancia_promedio_por_estado_F.items() if estado != 0}
ganancias_compañias['Ford'] = ganancia_promedio_por_estado_F

In [138]:
#honda data
honda_data = pd.read_csv('../CSV/Honda2.csv')
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
honda_data = honda_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(honda_data)):
    percent = (honda_data['Adj Close'][i] - honda_data['Adj Close'][i-1])/honda_data['Adj Close'][i-1]
    percentage.append(percent)
honda_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
honda_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
honda_data.insert(4, 'Current State', states)

ganancia_promedio_por_estado_H = honda_data.groupby('Current State')['Change %'].mean()
ganancia_promedio_por_estado_H = ganancia_promedio_por_estado_H.to_dict()
ganancia_promedio_por_estado_H = {estado: ganancia for estado, ganancia in ganancia_promedio_por_estado_H.items() if estado != 0}
ganancias_compañias['Honda'] = ganancia_promedio_por_estado_H

In [139]:
#Nio data
NIO_data = pd.read_csv('../CSV/NIO2.csv')
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
NIO_data = NIO_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(NIO_data)):
    percent = (NIO_data['Adj Close'][i] - NIO_data['Adj Close'][i-1])/NIO_data['Adj Close'][i-1]
    percentage.append(percent)
NIO_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
NIO_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
NIO_data.insert(4, 'Current State', states)

ganancia_promedio_por_estado_N = NIO_data.groupby('Current State')['Change %'].mean()
ganancia_promedio_por_estado_N = ganancia_promedio_por_estado_N.to_dict()
ganancia_promedio_por_estado_N = {estado: ganancia for estado, ganancia in ganancia_promedio_por_estado_N.items() if estado != 0}
ganancias_compañias['NIO'] = ganancia_promedio_por_estado_N


In [140]:
# Toyota data
toyota_data = pd.read_csv('../CSV/Toyota2.csv')
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
toyota_data = toyota_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(toyota_data)):
    percent = (toyota_data['Adj Close'][i] - toyota_data['Adj Close'][i-1])/toyota_data['Adj Close'][i-1]
    percentage.append(percent)
toyota_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
toyota_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
toyota_data.insert(4, 'Current State', states)

ganancia_promedio_por_estado_T = toyota_data.groupby('Current State')['Change %'].mean()
ganancia_promedio_por_estado_T = ganancia_promedio_por_estado_T.to_dict()
ganancia_promedio_por_estado_T = {estado: ganancia for estado, ganancia in ganancia_promedio_por_estado_T.items() if estado != 0}
ganancias_compañias['Toyota'] = ganancia_promedio_por_estado_T


In [141]:
print("Ganancias promedio por estado de cada compañía")
for company, ganancias in ganancias_compañias.items():
    print(f"Empresa: {company}")
    print(ganancias, "\n")

Ganancias promedio por estado de cada compañía
Empresa: Ford
{'B1': -0.011724641822417328, 'B2': -0.0324795043779447, 'B3': -0.06773767548147779, 'S1': 0.011535398981854862, 'S2': 0.03358753568170456, 'S3': 0.06488502390721564} 

Empresa: Honda
{'B1': -0.0071227418916923226, 'B2': -0.02167484233901611, 'B3': -0.03303890693678714, 'S1': 0.00698423151178288, 'S2': 0.021325549783340243, 'S3': 0.035134801493423524} 

Empresa: NIO
{'B1': -0.022890291659479898, 'B2': -0.06181073733774701, 'B3': -0.09680675287484451, 'S1': 0.020850650110335038, 'S2': 0.06767780090299169, 'S3': 0.13352620494181355} 

Empresa: Toyota
{'B1': -0.006558936289452121, 'B2': -0.01904192024289959, 'B3': -0.03211888144790053, 'S1': 0.006395304250082628, 'S2': 0.018578816609474934, 'S3': 0.037220573832227294} 



In [146]:
# ganancias esperadas con base en el comportamiento en el limite

def ganancias_esperadas_limite(transicion, ganancias, n):
    ganancias_esperada = 0
    distribucion_estacionaria = get_state_distribution(transicion, n)
    for i in range(n):
        ganancias_esperada += distribucion_estacionaria[i]*(ganancias[0][i]*100)
    return ganancias_esperada

for company, ganancias in ganancias_compañias.items():
    print(f"Ganancias esperadas en el límite para {company}:")
    print(ganancias_esperadas_limite(company_map[company], [list(ganancias.values())], 6), "\n")




Ganancias esperadas en el límite para Ford:
[0.1445789] 

Ganancias esperadas en el límite para Honda:
[0.06325776] 

Ganancias esperadas en el límite para NIO:
[0.19320589] 

Ganancias esperadas en el límite para Toyota:
[0.07980421] 

